In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MySecondPySpark').getOrCreate()

In [3]:
students_data = [
    ("John", "Math", 85),
    ("John", "Science", 90),
    ("Emily", "Math", 92),
    ("Emily", "Science", 88),
    ("Mike", "Math", 78),
    ("Mike", "Science", 85)
]
students_df = spark.createDataFrame(students_data, ["Name", "Subject", "Score"])

In [4]:
students_df.show()

+-----+-------+-----+
| Name|Subject|Score|
+-----+-------+-----+
| John|   Math|   85|
| John|Science|   90|
|Emily|   Math|   92|
|Emily|Science|   88|
| Mike|   Math|   78|
| Mike|Science|   85|
+-----+-------+-----+



In [5]:
### Find all students who scored more than 85 in any subject
students_df.filter(students_df['Score'] > 85).show()
# For more than or equal to 85
students_df.filter(students_df['Score'] >= 85).show()

+-----+-------+-----+
| Name|Subject|Score|
+-----+-------+-----+
| John|Science|   90|
|Emily|   Math|   92|
|Emily|Science|   88|
+-----+-------+-----+

+-----+-------+-----+
| Name|Subject|Score|
+-----+-------+-----+
| John|   Math|   85|
| John|Science|   90|
|Emily|   Math|   92|
|Emily|Science|   88|
| Mike|Science|   85|
+-----+-------+-----+



In [7]:
# 2. Calculate the average score for each subject
from pyspark.sql.functions import avg
students_df.groupBy("Subject").agg(avg("Score").alias("Average_Score")).show()

+-------+-----------------+
|Subject|    Average_Score|
+-------+-----------------+
|Science|87.66666666666667|
|   Math|             85.0|
+-------+-----------------+



In [11]:
# 3. Find which student has the highest total score across all subjects
from pyspark.sql.functions import sum
print("\nStudent with the highest total score:")
students_df.groupBy("Name").agg(sum("Score").alias("Total_Score")) \
 .orderBy("Total_Score", ascending=False) \
 .limit(1).show()


Student with the highest total score:
+-----+-----------+
| Name|Total_Score|
+-----+-----------+
|Emily|        180|
+-----+-----------+



In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, sum, when
# 4. Add a column "Grade" with:
#    - "A" if Score >= 90
#    - "B" if 80 <= Score < 90
#    - "C" if Score < 80
print("\nStudents with Grade column: ")
students_df = students_df.withColumn(
    "Grade",
    when(students_df['Score'] >= 90, "A")
    .when((students_df['Score'] >= 80) & (students_df['Score'] < 90), "B")
    .otherwise("C")
)
students_df.show()


Students with Grade column: 
+-----+-------+-----+-----+
| Name|Subject|Score|Grade|
+-----+-------+-----+-----+
| John|   Math|   85|    B|
| John|Science|   90|    A|
|Emily|   Math|   92|    A|
|Emily|Science|   88|    B|
| Mike|   Math|   78|    C|
| Mike|Science|   85|    B|
+-----+-------+-----+-----+

